In [3]:
import findspark
findspark.init()
from pyspark import SparkContext
import numpy as np

sc = SparkContext("local[*]", "Name of the Program")

25/02/24 16:32:17 WARN Utils: Your hostname, jalvarez resolves to a loopback address: 127.0.1.1; using 192.168.1.51 instead (on interface wlo1)
25/02/24 16:32:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/24 16:32:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/24 16:32:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [31]:
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,3545.301892,3198.013947,80.000155,1.000002,4.449601e+08,4.768073e+02,13.0,-7.710425e-09,87.000001,1.909943e+07,2.468368e+09,0
1,-0.000001,7.999989,65499.998878,1.000000,6.199989e+01,6.999981e+01,12.0,-2.356882e-08,8.000005,2.468370e+09,2.468368e+09,1
2,3599.999088,54943.060117,53.000260,1.000002,2.629992e+02,8.299989e+01,13.0,5.000000e+00,212.999999,1.590217e+09,9.830836e+01,0
3,13.270015,1359.530053,54976.967280,1.000002,6.415608e+08,8.299989e+01,12.0,3.000000e+00,7.999999,2.468370e+09,4.026532e+09,0
4,1242.891867,7.999201,30867.053528,1.000002,6.462429e+08,8.205111e+02,13.0,5.000000e+00,131.433345,1.399758e+09,4.026532e+09,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.000001,1592.529246,24.999998,1.000000,3.828049e+04,6.999981e+01,13.0,-2.356882e-08,-1.000002,2.468370e+09,3.651344e+09,1
9996,932.759112,55642.021416,21309.076471,1.000000,3.449995e+02,2.118172e+07,13.0,-2.356882e-08,199.000002,2.468370e+09,3.495627e+07,1
9997,3599.999088,50465.136134,0.000351,2.000000,2.629992e+02,8.299989e+01,13.0,5.000000e+00,212.999999,1.043512e+09,9.830836e+01,0
9998,2041.282575,58156.737042,-0.000044,2.000000,6.199989e+01,6.999981e+01,13.0,4.000000e+00,187.000001,2.468370e+09,2.151554e+09,1


In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
data=pd.read_csv("data/botnet_reduced_10k_l.csv", header=None,sep=',')
r=np.array(data)[:,:-1]
print(r.shape)
scaler = StandardScaler()
r=scaler.fit_transform(r)

(10000, 11)


In [35]:
r

array([[ 1.41281668, -0.75956071, -0.41204268, ..., -0.40163083,
        -2.91039199,  0.1567098 ],
       [-0.79867306, -0.89146231,  3.64034447, ..., -1.27444226,
         0.4762026 ,  0.1567098 ],
       [ 1.44693571,  1.38000803, -0.41371517, ...,  0.9904482 ,
        -0.73801555, -1.74828175],
       ...,
       [ 1.44693571,  1.19485357, -0.4169982 , ...,  0.9904482 ,
        -1.49394086, -1.74828175],
       [ 0.47463895,  1.51288805, -0.41699823, ...,  0.70319381,
         0.47620254, -0.08779544],
       [-0.2312852 ,  1.28112832, -0.41544962, ...,  0.70319381,
         0.4762026 ,  0.15671301]], shape=(10000, 11))

In [203]:
def readFile(filename):
    rdd = sc.textFile(filename)
    rdd_numpy = rdd.map(lambda x : np.array(x.split(","), dtype = float))
    rdd_return = rdd_numpy.map(lambda x : (x[:-1], int(x[-1])))
    return rdd_return

def normalize(RDD_Xy):
    
    suma = RDD_Xy.reduce(lambda x,y: (np.array(x[0])+np.array(y[0]),x[1]))
    total_filas = float(RDD_Xy.count())
    media = suma[0] / total_filas

    rdd_2 = RDD_Xy.map(lambda x: (x[0],x[1], media)) 

    restas = rdd_2.map(lambda x: (x[0]-x[2])**2)
    sumas = restas.reduce(lambda x,y: x+y)
    desviacion = (sumas / total_filas)**0.5

    def f(x):
        a = (x[0] - media) / desviacion
        a= np.nan_to_num(a, copy=False,nan=0.0)
        return (a,x[1])

    normalizado = RDD_Xy.map(f)
    return normalizado

def predict(w,b,X,threshold=0.5):
    rdd=X.map(lambda x:np.dot(w,np.array(x[0])+b))
    logistic= rdd.map(lambda x: 1/(1+np.exp(-x)))
    results=logistic.map(lambda x: 1 if x>threshold else 0)
    return results

def get_predictions_targets_inputs(w,b,X,threshold=0.5):
    """
    Get the predictions, targets and inputs
    :param w: weights
    :param b: bias
    :param X: RDD with the data
    :param threshold: threshold to classify the data
    :return: RDD with the tuple predictions, targets and inputs
    """
    rdd=X.map(lambda x:(np.dot(w,np.array(x[0])+b),x[1],x[0]))
    logistic= rdd.map(lambda x: (1/(1+np.exp(-x[0])),x[1],x[2]))
    results=logistic.map(lambda x: ((0.999,x[1],x[2]) if x[0]>threshold else (0.001,x[1],x[2]))) #para evitar log(0)=-inf y log(1)=0
    return results

# TODO preguntar si la b se incluye en la perdida, porque en la formula del enunciado no aparece

def calculate_loss_function(predictions_and_targets,lambda_reg,w):   
    """
    Calculate the loss function for logistic regression
    :param predictions_and_targets: RDD with the predictions and the targets
    :param lambda_reg: regularization parameter
    :param w: weights
    :return: loss function
    """


    def f(x):
        return (-x[1]*np.log(x[0])-(1-x[1])*np.log(1-x[0]))

    loss=predictions_and_targets.map(f)
    suma=loss.reduce(lambda x,y: x+y)/predictions_and_targets.count()
    squared_w=np.dot(w,w)
    squared_w=squared_w*lambda_reg/(2*len(w))
    return suma+squared_w



# TODO pregunta si el for de los gradientes se puede hacer así

def train(RDD_Xy, iterations, learning_rate, lambda_reg):
    w = np.random.rand(11)
    b=np.random.rand(1)
    for iteration in range(iterations):
        predictions_targets_inputs=get_predictions_targets_inputs(w,b,RDD_Xy)
        for i in range(len(w)):
            gradient=predictions_targets_inputs.map(lambda x: (x[0]-x[1])*x[2][i]).reduce(lambda x,y: x+y)/predictions_targets_inputs.count()
            gradient=gradient+(lambda_reg*w[i])/len(w)
            w[i]=w[i]-learning_rate*gradient
        b_gradient=predictions_targets_inputs.map(lambda x: x[0]-x[1]).reduce(lambda x,y: x+y)/predictions_targets_inputs.count()
        b=b-learning_rate*b_gradient
        loss=calculate_loss_function(predictions_targets_inputs,lambda_reg,w)
        print("Iteration: ",iteration," Loss: ",loss)

    return w,b

def accuracy(w,b,RDD_Xy):
    predictions_and_targets=get_predictions_targets_inputs(w,b,RDD_Xy)
    correct=predictions_and_targets.map(lambda x: 1 if round(x[0])==x[1] else 0)
    correct=correct.reduce(lambda x,y: x+y)
    return correct/RDD_Xy.count()

In [193]:
n = readFile('data/botnet_reduced_10k_l.csv')

In [194]:
normalizado = normalize(n)

In [207]:
w,b=train(normalizado,10,1.5,0)

Iteration:  0  Loss:  2.399025759480246


Iteration:  1  Loss:  1.3360761990462704


Iteration:  2  Loss:  0.8305017492493122


Iteration:  3  Loss:  0.6909853027205992


Iteration:  4  Loss:  0.5991254641645511
Iteration:  5  Loss:  0.5452527768910823


Iteration:  6  Loss:  0.5100283275199671


Iteration:  7  Loss:  0.495524142484798
Iteration:  8  Loss:  0.4761852291045759
Iteration:  9  Loss:  0.46099036859154585


In [208]:
acc=accuracy(w,b,normalizado)

In [209]:
print(f"accuracy: {acc}")

accuracy: 0.9334
